##### Grabbing our file

We test this on memo from all four different presidents.

In [1]:
import os.path
import io
save_path = "C:/Users/avish/OneDrive/Documents/Presidential-Transcripts/Files"

In [2]:
import pandas as pd
import numpy as np
import re

###### Title

In [3]:
def gettitle(text):
    title = re.findall('Memorandum on[^\n]*', text)
    if len(title) == 0:
        text = re.sub('\n\n', 'SPLIT-HERE', text)
        title = re.findall('Memorandum [A-Z][\W\w]*?SPLIT-HERE', text)[0]
        title = re.sub('\n', '', title)
        title = re.sub('SPLIT-HERE', '', title)
        text = re.sub('SPLIT-HERE', '\n\n', text)
    else: 
        title = title[0]
    return title

###### Subject

In [4]:
def getsubject(text):
    subject = re.findall('Subject:\n <\/i>\n [^\n]*', text)
    if len(subject) == 0:
        text = re.sub('\n\n', 'SPLIT-HERE', text)
        subject = re.findall('Subject: [A-Z][\W\w]*?SPLIT-HERE', text)
        if len(subject) == 0:
            subject = np.nan
            text = re.sub('SPLIT-HERE', '\n\n', text)
            return subject
        subject = re.sub('\n', '', subject[0])
        subject = re.sub('SPLIT-HERE', '', subject)
        text = re.sub('SPLIT-HERE', '\n\n', text)
    else: 
        subject = subject[0]
        subject = re.sub('\n', '', subject)
        subject = re.sub('<\/i>', '', subject)
        subject = re.sub('  ', ' ', subject)
    subject = re.findall('Subject: (.*)', subject)[0]
    return subject

###### Memorandum For

In [5]:
def getmemfor(text):
    text = re.sub('\n\n', 'SPLIT-HERE', text)
    memfor = re.findall('Memorandum for[\W\w]*?SPLIT-HERE', text)
    if len(memfor) == 0:
        memfor = re.findall('Memorandum for [^\n]*', text)[0]
    else: 
        memfor = memfor[0]
        memfor = re.sub('\n', '', memfor)
        memfor = re.sub('SPLIT-HERE', '', memfor)
    text = re.sub('SPLIT-HERE', '\n\n', text)
    return memfor

###### Subjects

In [6]:
def getsubjects(text):
    subjects = re.findall('Subjects:\n.*\n[^\n]*', text)
    if len(subjects) == 0:
        subjects = np.nan
    else:
        subjects = subjects[0]
        subjects = re.sub('\n', '', subjects)
        subjects = re.sub('<\/i>', '', subjects)
        subjects = re.sub('  ', ' ', subjects)
        subjects = re.findall('Subjects: (.*)', subjects)[0]
    return subjects

###### Notes

In [7]:
def getnote(text):
    note = re.findall('This memorandum[^.]*', text)
    if len(note) == 0:
        note = re.findall('[Nn]\n  <font size="\+1">\n   [Oo][Tt][Ee]\n   <font size="\+1">\n.*', text)
        if len(note) == 0:
            note = np.nan
        else:
            note = re.findall(':(.*)', note[0])[0]
    else:
        note = note[0]
        note = re.sub('\\n|<i>|<\/i>', '', note)
        note = re.sub('       |      ', ' ', note)
        note = note + '.'
    return note

###### Categories

In [8]:
def getcategories(text):
    categories = re.findall('(?<=Categories.\n <\/i>\n ).*(?=\n)', text)
    if len(categories) == 0:
        categories = re.findall('(?<=Categories.\n  <\/i>\n ).*(?=\n)', text)
    if len(categories) == 0:
        categories = np.nan
    else:
        categories = categories[0]
    return categories

###### CPD Number

In [9]:
def getcpd(text):
    cpd = re.findall('(?<=DCPD).*?(?=\.)', name)
    if len(cpd) == 0:
        cpd = re.findall('WCPD\d*', name)[0]
        cpd = re.sub('WCPD', '', cpd)
    else:
        cpd = cpd[0]
    return cpd

###### Date

In [10]:
def getdate(text):
    date = re.findall("(?<=p>\\n<i>\\n ).*?(?= ) [\d].*?, \d\d\d\d", text)
    if len(date) == 0:
        date = re.findall('\n\n.*?\n\n', text)
        if len(date) == 0:
            date = re.findall('[A-Z][a-z]*? \d\d?, \d\d\d\d', text)[0]
        else:
            date = date[0]
        date = re.sub('\n\n', '', date)
        if date[0] == ' ':
            date = date[1:]
    else:
        date = date[0]
    return date

###### Date Filed

In [11]:
def getfiled(text):
    filed = re.findall('\[.*\]', text)
    if len(filed) == 0 or len(re.findall('Filed', filed[0])) == 0:
        filed = np.nan
    else: 
        filed = filed[0]
        filed = filed[1:len(filed)-1]
        filed = re.findall('[\d].*', filed)[0]
    return filed

###### Administration

In [12]:
def getadmin(text):
    administration = re.findall('(?<=Administration of ).*?(?=,)', text)
    if len(administration) == 0:
        administration = re.findall('William J. Clinton', text)
        if len(administration) == 0:
            administration = re.findall('George W. Bush', text)
            if len(administration) == 0:
                administration = re.findall('George Bush', text)
                if len(administration) == 0:
                    administration = re.findall('Bill Clinton', text)[0]
                else:
                    administration = administration[0]
            else:
                administration = administration[0]
        else:
            administration = administration[0]
    else:
        administration = administration[0]
    return administration

###### Getting all of our memos

In [15]:
memos = []
names = []
for file in os.listdir(save_path):
    name = os.path.join(save_path, file)
    try:
        with open(name, "r") as f:
            text = f.read()
    except:
        pass
    try:
        getmemfor(text)
        gettitle(text)
        memos.append(text)
        names.append(file)
        break
    except:
        pass

For some reason, remarks failed to open.  This is not important right now though.

In [16]:
coltitle = []
colsubj = []
colmemfor = []
colsubjs = []
colnotes = []
colcateg = []
colcpdnum = []
coldate = []
coldatefile = []
coladmin = []
for text in memos:
    coltitle.append(gettitle(text))
    colsubj.append(getsubject(text))
    colmemfor.append(getmemfor(text))
    colsubjs.append(getsubjects(text))
  #  colnotes.append(getnote(text))
    colcateg.append(getcategories(text))
    coldate.append(getdate(text))
    coldatefile.append(getfiled(text))
    coladmin.append(getadmin(text))
for name in names:
     colcpdnum.append(getcpd(name))

<html>
 <head>
  <title>
   Weekly Compilation of Presidential Documents Volume 29 Issue 3 (Monday, January 25, 1993)
  </title>
 </head>
 <body>
  <pre>[Weekly Compilation of Presidential Documents Volume 29, Number 3 (Monday, January 25, 1993)]
[Pages 64-65]
[Online from the Government Publishing Office, <a href="http://www.gpo.gov">www.gpo.gov</a>]

&lt;R04&gt;
Memorandum on Assistance to Refugees of Tajikistan

 January 19, 1993

Presidential Determination No. 93-14

Memorandum for the Secretary of State

Subject: Determination Pursuant to Section 2(c)(1) of the Migration and 
Refugee Assistance Act of 1962, as Amended

    Pursuant to section 2(c)(1) of the Migration and Refugee Assistance 
Act of 1962, as amended, 22 U.S.C. 2601(c)(1), I hereby determine that 
it is important to the national interest that up to $5,000,000 be made 
available from the U.S. Emergency Refugee and Migration Assistance Fund 
to meet the urgent and unexpected needs of refugees, displaced persons, 
and v

In [34]:
memotable = pd.DataFrame()
memotable['Title'] = coltitle
memotable['Memorandum For'] = colmemfor 
memotable['Subject'] = colsubj
#memotable['Note'] = colnotes
memotable['Categories'] = colcateg
memotable['Subjects'] = colsubjs
memotable['Administration'] = coladmin
memotable['Date'] = coldate
memotable['Date Filed'] = coldatefile
memotable['CPD Number'] = colcpdnum


In [35]:
memotable

,Title,Memorandum For,Subject,Categories,Subjects,Administration,Date,Date Filed,CPD Number
0,Memorandum on Assistance to Refugees of Tajiki...,Memorandum for the Secretary of State,Determination Pursuant to Section 2(c)(1) of t...,NaN,NaN,George Bush,"January 19, 1993",NaN,19930125
1,Memorandum on Fetal Tissue Transplantation Res...,Memorandum for the Secretary of Health and Hum...,Federal Funding of Fetal Tissue Transplantatio...,NaN,NaN,William J. Clinton,"January 22, 1993",NaN,19930125
2,Memorandum on the Title X ``Gag Rule'',Memorandum for the Secretary of Health and Hum...,The Title X ``Gag Rule'',NaN,NaN,William J. Clinton,"January 22, 1993",NaN,19930125
3,Memorandum on the Mexico City Policy,Memorandum for the Acting Administrator of the...,AID Family Planning Grants/Mexico City Policy,NaN,NaN,William J. Clinton,"January 22, 1993",NaN,19930125
4,Memorandum on Abortions in Military Hospitals,Memorandum for the Secretary of Defense,Privately Funded Abortions at Military Hospitals,NaN,NaN,William J. Clinton,"January 22, 1993",NaN,19930125
5,Memorandum on Importation of RU-486,Memorandum for the Secretary of Health and Hum...,Importation of RU-486,NaN,NaN,William J. Clinton,"January 22, 1993",NaN,19930125
6,Memorandum on Ending Discrimination in the Arm...,Memorandum for the Secretary of Defense,Ending Discrimination on the Basis of Sexual O...,NaN,NaN,William J. Clinton,"January 29, 1993",NaN,19930201
7,Memorandum on Review of Regulations,Memorandum for the Acting Director,NaN,NaN,NaN,William J. Clinton,"January 21, 1993",NaN,19930201
8,Memorandum on Delegation of Authority Concerni...,Memorandum for the Secretary of State,Delegation of Authority With Respect to Report...,NaN,NaN,William J. Clinton,"February 3, 1993",NaN,19930208
9,Memorandum on Fiscal Responsibility,Memorandum for the Heads of Executive Departme...,Government Fiscal Responsibility and Reducing ...,NaN,NaN,William J. Clinton,"February 10, 1993",NaN,19930215


In [37]:
memotable.to_csv('memo.csv')

<bound method Tag.prettify of Memorandum on Assistance to Refugees of Tajikistan

 January 19, 1993

Presidential Determination No. 93-14

Memorandum for the Secretary of State

Subject: Determination Pursuant to Section 2(c)(1) of the Migration and 
Refugee Assistance Act of 1962, as Amended

    Pursuant to section 2(c)(1) of the Migration and Refugee Assistance 
Act of 1962, as amended, 22 U.S.C. 2601(c)(1), I hereby determine that 
it is important to the national interest that up to $5,000,000 be made 
available from the U.S. Emergency Refugee and Migration Assistance Fund 
to meet the urgent and unexpected needs of refugees, displaced persons, 
and victims of conflict from Tajikistan. These funds may be contributed 
on a multilateral or bilateral basis as appropriate to international 
organizations, private voluntary organizations, and other governmental 
and nongovernmental organizations engaged in this relief effort.
    You are directed to inform the appropriate committees of t